In [ ]:
# running this once for the first time.
!mkdir -p downloads intermediates outputs

# download tag achives from nyanko7/danbooru2023
!wget -q --show-progress -c -P downloads/ "https://huggingface.co/datasets/nyanko7/danbooru2023/resolve/main/metadata/artists.tar.xz"
!wget -q --show-progress -c -P downloads/ "https://huggingface.co/datasets/nyanko7/danbooru2023/resolve/main/metadata/tags.tar.xz"

# untar
!tar xf downloads/artists.tar.xz -C intermediates/
!tar xf downloads/tags.tar.xz -C intermediates/

In [ ]:
# we only need 3 columns.
!sed -e 's/^.*antecedent_name":"\([^"]*\).*consequent_name":"\([^"]*\).*status":"\([^"]*\).*$/\1, \2, \3/g' intermediates/tag_aliases.json > intermediates/tag_aliases.csv
!sed -e 's/^.*name":"\(.*\)","post_count":\([0-9]*\),"category":\([0-9]*\).*$/\1, \3, \2/' intermediates/tags.json > intermediates/tags.csv

In [ ]:
import re
import sys

IGNORE_ALIAS_STATES = [ 'retired', 'deleted' ]

# get aliases
file = "intermediates/tag_aliases.csv"
alias_dict = {}
with open(file, 'r') as infile:
    for row in infile:
        antecedent, consequent, status = row.replace('\\', '').split(', ')
        if status in IGNORE_ALIAS_STATES:
            continue
        consequent = consequent.strip().replace(' ', '_')
        antecedent = antecedent.strip().replace(' ', '_')
        if consequent not in alias_dict:
            alias_dict[consequent] = []
        alias_dict[consequent].append(antecedent)
infile.close()

#check
dict(list(alias_dict.items())[:10])

In [ ]:

# read data from taglist and store to csv
file = "intermediates/tags.csv"
unique_data = {}
with open(file, 'r') as infile:
    for row in infile:
        try:
            name, category, count = row.strip().split(', ')
            if int(count) > 0:
                unique_data[name] = { "category": int(category), "count": int(count) }
        except:
            print(row)
            raise Exception("nahhh")
infile.close()

ordered_data = dict(sorted(unique_data.items(), key=lambda x: x[1]["count"], reverse=True))

with open("outputs/danbooru2023.csv", 'w') as outfile:
    for key, value in ordered_data.items():
        key_to_show = f'"{key}"' if "," in key else key
        outfile.write(f"{key_to_show},{value['category']},{value['count']}")
        if (key in alias_dict):
            outfile.write(f',"{",".join(alias_dict[key])}"')
        outfile.write("\n")
outfile.close()